In [2]:
%%html
<style>
table {float:left}
</style>

# **Working with HPE IMC API for Operators**

In this notebook, we will be covering the basics of using the pyhpimc python module to access the RESTFUL interface ( *eAPI* ) of the HPE IMC Network Management Server. 

The python library is currently available at [HPE Github repository](https://github.com/HPENetworking/HP-Intelligent-Management-Center/blob/master/README.md)

## Import Required Modules from pyhpeimc
Before we get started, we must import the required libraries into the ipython interpreter . For this notebook, you will require the Auth and Operator modules from the pyhpimc library, as well as the standard csv module. 

In [5]:
import csv
import time
from pyhpeimc.auth import *
from pyhpeimc.plat.operator import *

## Input IMC Credentials

In this section, we will first create a authentication object which will contain

- the protocol
- ip address
- port number
- username
- password

If you are running this on your local network, please change the appropriate values in the following field to connect to your local IMC server. 

In [6]:
auth = IMCAuth("http://", "10.101.0.203", "8080", "admin", "admin")


## List Operators
In this step we list the current operators that exist in the system by using the **get_plat_operators** function and capturing the output into a varible called *ops_list*.

We will then run the **len** which measures the length of the ops_list object, or more simply, measures the number of operators currently configured in the system.

In [7]:
ops_list = get_plat_operator(url=auth.url, auth=auth.creds)
print ("There are currently " + str(len(ops_list)) + " operators configured.")

There are currently 9 operators configured.


Shown here is a screen capture of the current operators configured in the HPE IMC system. You can see that there are the same amoutn of operators as shown in the statement above. 

*note: The screen capture is used for the intial demonstration only. If you are running this notebook against your own IMC server, this screen capture may not match what you have configured on your system.*

![Show Operators](./images/operators1.png "Show Operators")

## Importing Operators

In the following section, we are going to import a group of operators from a prepared CSV file. 

For more information on the contents of the CSV file, please see the documentation available at http://ip_address:8080/imcrs  on your local IMC server. 

The CSV file provided is for example only. Please feel free to use it as a template, but you are encouraged to look at the documentation for more information on the specific attributes of the CSV file. 

This file contains the following operators

| Operator | Fullname |
| ------ | ----------- |
| cyoung   | Christoper Young |
| lstaurt | Les Stuart |
| kgott    | Ken Gott |
| rkauffman | Rick Kauffman |
| lknapp | Leonard Knapp |
| clinzell | Chris Linzell |
| dwenger | Dwayne Wenger |
| jmacdonald | James Macdonald |
| tkubica | Tomas Kubica |
| mbreen | Michael Breen|
| mventer | Marius Venter |

In [9]:
with open('imc_operator_list.csv') as f:
    s = f.read()
    print (s)

fullName,sessionTimeout,password,operatorGroupId,name,desc,defaultAcl,authType
Christopher Young,10,access4chris,1,cyoung,Chris account,0,0
Les Stuart,10,access4les,1,lstuart,admin account,0,0
Ken Gott,10,access4ken,1,kgott,admin account,0,0
Rick Kauffman,10,access4rick,1,rkauffman,admin account,0,0
Leonard Knapp,10,access4leonard,1,lknapp,admin account,0,0
Chris Linzell,10,access4chris,1,clinzell,admin account,0,0
Dwayne Wenger,10,access4dwayne,1,dwenger,admin account,0,0
James Macdonald,10,access4jim,1,jmacdonald,admin account,0,0
Tomas Kubica,10,access4tomas,1,tkubica,admin account,0,0
Michael Breen,10,access4michael,1,mbreen,admin account,0,0
Marius Venter,10,access4marius,1,mventer,marius admin account,0,0
Kevin Secino,10,access4kevin,1,ksecino,marius admin account,0,0



# Create import_operator function

In this section, we have created a simple python function to perform the following tasks

- Use the standard library CSV module to open the file **imc_operator_list.csv** file. 
- Use a standard *For* loop to take each line from the CSV file and run the **create_operator** function from the pyhpimc library over it.

This function will allow us to run the **create_operator** function once for each operator in the csv file.

In [10]:
def import_operator(filename='imc_operator_list.csv'):
    with open(filename) as csvfile:
        # decodes file as csv as a python dictionary
        reader = csv.DictReader(csvfile)
        try:
            for operator in reader:
                # loads each row of the CSV as a JSON string
                create_operator(operator, url=auth.url, auth=auth.creds)
        except:
            pass

## Run Import Operators Function

We will now run the **import_operator** function we created above. In this example, we do not have to input the name of the csv file as we are using the default values as configured in the function (*filename='imc_operator_list.csv'*)

In [11]:
start_time = time.time()
import_operator()
print("--- %s seconds ---" % (time.time() - start_time))

Operator Successfully Created
Operator Successfully Created
Operator Successfully Created
Operator Successfully Created
Operator Successfully Created
Operator Successfully Created
Operator Successfully Created
Operator Successfully Created
Operator Successfully Created
Operator Successfully Created
Operator Successfully Created
Operator Successfully Created
--- 59.5469970703125 seconds ---


## List Operators

We will now list the number of operators in the system. 

In [9]:
ops_list = get_plat_operator(url=auth.url, auth=auth.creds)
print ("There are currently " + str(len(ops_list)) + " operators configured.")

There are currently 12 operators configured.


Shown here is a screen capture of the current operators configured in the HPE IMC system. You can see that there are the same amoutn of operators as shown in the statement above. 

*note: The screen capture is used for the intial demonstration only. If you are running this notebook against your own IMC server, this screen capture may not match what you have configured on your system.*

![Show Operators](./images/operators2.png "Show Operators")

# Resetting an Operator Password
We will now use the **set_operator_password** function to set the password of the newly created **cyoung** account to something a little more secure. 

Feel free to try using this function to reset the password of another account by replacing the *cyoung* variable in the function below.

In [10]:
set_operator_password('cyoung', password='newpass',auth=auth.creds,url=auth.url,)
set_operator_password('')


 Operator:cyoung password was successfully changed


TypeError: set_operator_password() missing 3 required positional arguments: 'password', 'auth', and 'url'

# Clean up the system when we're done

In the following section, we will continue to use the API to clean up the system, using the *delete_plat_operator* function to delete all the new operators.

We will
- create a new python list called ops_list with the name of all the operators we configured as items in the list. 
- Use a standard *For* loop to take each line from the ops_list object and run the **delete_plat_operator** function from the pyhpimc library over it.


In [3]:
ops_list = ['cyoung', 'lstuart', 'clinzell', 'kgott', 'rkauffman', 'lknapp', 'dwenger', 'jmacdonald', 'tkubica', 'mbreen', 'mventer']

In [4]:
for operator in ops_list:
    delete_plat_operator(operator,url=auth.url, auth=auth.creds, headers=HEADERS)


 Operator: cyoung was successfully deleted

 Operator: lstuart was successfully deleted

 Operator: clinzell was successfully deleted

 Operator: kgott was successfully deleted

 Operator: rkauffman was successfully deleted

 Operator: lknapp was successfully deleted

 Operator: dwenger was successfully deleted

 Operator: jmacdonald was successfully deleted

 Operator: tkubica was successfully deleted

 Operator: mbreen was successfully deleted

 Operator: mventer was successfully deleted
